In [1]:
import torch
import matplotlib.pyplot as plt
import plotly.express as px
import sys
import glob
import json
import numpy as np
from tqdm import tqdm
sys.path.append("../")

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import glob
import os
import pandas as pd

def get_npy_files(root_path):
    # Use a glob pattern to match all .npz files in the directory and subdirectories
    pattern = os.path.join(root_path, '**', '*.npz')
    npy_files = glob.glob(pattern, recursive=True)
    return npy_files

all_npy_files = get_npy_files("/app/dataset/npz_all/npz/layout")+get_npy_files("/app/dataset/npz_all/npz/tile/xla")
print(len(all_npy_files))
# Create a DataFrame from the list of npy files
df = pd.DataFrame(all_npy_files, columns=["path"])
print(len(df))
def find_split(x):
    if "/train" in x:
        return "train"
    elif "/valid" in x:
        return "valid"
    elif "/test" in x:
        return "test"
    return None
def find_configtype(x):
    if "/default" in x:
        return "default"
    elif "/random" in x:
        return "random"
    return "istile"
def find_modeltype(x):
    if "/nlp" in x:
        return "nlp"
    elif "/xla" in x:
        return "xla"
    return None
def find_grapthtype(x):
    if "/layout" in x:
        return "layout"
    elif "/tile" in x:
        return "tile"
    return None
df["split"] = df.path.apply(lambda x : find_split(x))
df["config_type"] = df.path.apply(lambda x : find_configtype(x))
df["modeltype"] = df.path.apply(lambda x : find_modeltype(x))
df["graphtype"] = df.path.apply(lambda x : find_grapthtype(x))
df["modelname"] = df.path.apply(lambda x: x.split("/")[-1])

7868
7868


In [3]:
path = df[(df.modeltype=="xla")&(df.graphtype=="layout")&(df.split=="train")].path.to_list()[0]

In [4]:
data = dict(np.load(path))

In [10]:
import networkx as nx
from pyvis.network import Network
import numpy as np

# Initialize a networkx graph
G = nx.Graph()

# Add nodes with attributes to the graph, converting opcodes to int (JSON serializable)
for idx, opcode in enumerate(data['node_opcode']):
    G.add_node(int(idx), opcode=int(opcode))

# Add edges to the graph, making sure each ID is an integer
edges = data['edge_index']
for edge in edges:
    G.add_edge(int(edge[0]), int(edge[1]))

# Initialize PyVis network
nt = Network(height="800px", width="100%", notebook=False, heading='Graph Visualization')

colormap = plt.get_cmap('nipy_spectral')
# Normalize opcode numbers to fit within the range of the colormap

norm = plt.Normalize(min(data['node_opcode']), max(data['node_opcode']))

# Map opcodes to colors using the colormap
opcode_colors = {opcode: colormap(norm(opcode)) for opcode in data['node_opcode']}

# Take the networkx graph and translate it to a pyvis graph
nt.from_nx(G)

# Set options for nodes
for node in nt.nodes:
    opcode = G.nodes[node['id']]['opcode']
    color = matplotlib.colors.rgb2hex(opcode_colors[opcode])  # Convert RGBA to hex
    node["title"] = f"Opcode: {opcode}"
    node["value"] = 10  # You can set this to represent a specific metric
    node["label"] = ''  # You can remove or set to some label you want to show
    node["color"] = color

nt.set_options("""
{
  "nodes": {
    "font": {
      "size": 12
    }
  },
  "edges": {
    "color": {
      "inherit": true
    },
    "smooth": false
  },
  "physics": {
    "hierarchicalRepulsion": {
      "centralGravity": 0.0,
      "springLength": 100,
      "springConstant": 0.01,
      "nodeDistance": 120,
      "damping": 0.09
    }
  }
}
""")

# Save the graph to an HTML file
nt.write_html('graph.html')
